In [1]:
import os
import sys
import keras
import numpy as np
import pandas as pd
from keras.layers import Dense
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from keras.models import Sequential, Model
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

import keras
from keras import layers
from keras import backend as K
from sklearn.model_selection import train_test_split


In [ ]:
args = parser.parse_args()
DATA_PATH = args.DATA_PATH
intermediate_dim = args.intermediate
latent_dim = args.latent_dim
epochs = args.epochs
batch_size = args.batch_size

# Example

In [10]:
DATA_PATH = "/Users/tgn531/Desktop/CBPP_22012021/Matrices_examples/GSE75748_sc_cell_type_ec.tsv"
latent_dim = 100
intermediate_dim = 500
epochs = 2
batch_size = 32

# Loads the data and preprocesses it

In [3]:
def load_data(DATA_PATH):
    """Loads the data and preprocesses it."""
    row_names = []
    array = []
    with open(DATA_PATH, 'r', encoding='utf-8') as infile:
        next(infile)
        for line in infile:
            line = line.split("\t")
            row_names.append(line[0])
            array.append(line[1:])

    expr = np.asarray(array, dtype=np.float32)
    expr = np.log2(1+expr[:])
    expr = expr / np.max(expr, axis=1, keepdims=True)
    expr = np.nan_to_num(expr)
    return expr, row_names

# Network Dimensions

In [4]:
x, row_names = load_data(DATA_PATH)
original_dim = x.shape[1]

# Sampling

In [5]:
def sampling(args):
    z_mean, z_log_sigma = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim),
                              mean=0., stddev=0.1)
    return z_mean + K.exp(z_log_sigma) * epsilon

In [6]:
inputs = keras.Input(shape=(original_dim,))
outputs = layers.Dense(original_dim, activation='sigmoid')(x)
opt = tf.keras.optimizers.Adam(learning_rate=0.001, clipnorm=0.001)


In [8]:
class VAE(keras.Model):
    def __init__(self, original_dim=original_dim, epochs = 100, intermediate_dim = 500, latent_dim = 100):
        super(VAE, self).__init__()
        inputs = keras.Input(shape=(original_dim,))
        h = layers.Dense(intermediate_dim, activation='relu')(inputs)
        
        z_mean = layers.Dense(latent_dim)(h)
        z_log_sigma = layers.Dense(latent_dim)(h)
        z = layers.Lambda(sampling)([z_mean, z_log_sigma])

        # Create encoder
        encoder = keras.Model(inputs, [z_mean, z_log_sigma, z], name='encoder')
        self.encoder = encoder
        # Create decoder
        latent_inputs = keras.Input(shape=(latent_dim,), name='z_sampling')
        x = layers.Dense(intermediate_dim, activation='relu')(latent_inputs) #relu
        
        
        outputs = layers.Dense(original_dim, activation='sigmoid')(x)
        decoder = keras.Model(latent_inputs, outputs, name='decoder')
        self.decoder = decoder
        
        # instantiate VAE model
        outputs = decoder(encoder(inputs)[2])
        vae = keras.Model(inputs, outputs, name='vae_mlp')
        
        # loss
        reconstruction_loss = keras.losses.mean_squared_error(inputs, outputs)
        reconstruction_loss *= original_dim        
        kl_loss = 1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma)
        kl_loss = K.sum(kl_loss, axis=-1)
        kl_loss *= -0.5   
        vae_loss = K.mean(0.9*(reconstruction_loss) + 0.1*(kl_loss))
        vae.add_loss(vae_loss)
        
        opt = tf.keras.optimizers.Adam(learning_rate=0.001 ,clipnorm=0.001)

        vae.compile(optimizer= opt, loss='mean_squared_error', metrics=['accuracy'])
        vae.fit(x_train, x_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, x_test))

# Train the VAE

In [11]:
x_train = x_test = np.array(x) 
vae = VAE(intermediate_dim = intermediate_dim, latent_dim = latent_dim, epochs=epochs)

Epoch 1/2
597/597 [==============================] - 7s 11ms/step - loss: 36.3529 - accuracy: 0.0043 - val_loss: 29.6568 - val_accuracy: 0.0076
Epoch 2/2
597/597 [==============================] - 7s 11ms/step - loss: 28.6775 - accuracy: 0.0095 - val_loss: 28.1405 - val_accuracy: 0.0095
